Displaying volume time series


In [1]:
from cluster import Cluster, Tier, bandwidth_share_model, compute_share_model, get_tier, convert_size
from phase import DelayPhase, ComputePhase, IOPhase
from application import Application
import simpy
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from analytics import display_cluster, display_apps, display_run
import numpy as np
from itertools import groupby
from operator import itemgetter
from loguru import logger
import itertools


In [ ]:

env = simpy.Environment()
data = simpy.Store(env)
nvram_bandwidth = {'read':  {'seq': 780, 'rand': 760},
                           'write': {'seq': 515, 'rand': 505}}
ssd_bandwidth = {'read':  {'seq': 210, 'rand': 190},
                    'write': {'seq': 100, 'rand': 100}}

ssd_tier = Tier(env, 'SSD', bandwidth=ssd_bandwidth, capacity=200e9)
nvram_tier = Tier(env, 'NVRAM', bandwidth=nvram_bandwidth, capacity=80e9)
cluster = Cluster(env, tiers=[ssd_tier, nvram_tier])
app1 = Application(env, name="app1", compute=[0, 10], read=[0, 0], write=[0, 5e9], data=data)
app2 = Application(env, name="app2", compute=[0, 15], read=[0, 0], write=[0, 0], data=data)
env.process(app1.run(cluster, tiers=[0, 0]))
env.process(app2.run(cluster, tiers=[1, 1]))
env.run()

fig = display_apps(data, height=400)
fig.show()
fig = display_cluster(data, cluster, height=600)
fig.show()

Displaying Cluster state

In [2]:
# remove logging


env = simpy.Environment()
data = simpy.Store(env)
nvram_bandwidth = {'read':  {'seq': 780, 'rand': 760},
                           'write': {'seq': 515, 'rand': 505}}
ssd_bandwidth = {'read':  {'seq': 210, 'rand': 190},
                    'write': {'seq': 100, 'rand': 100}}

ssd_tier = Tier(env, 'SSD', bandwidth=ssd_bandwidth, capacity=50e9)
nvram_tier = Tier(env, 'NVRAM', bandwidth=nvram_bandwidth, capacity=20e9)
cluster = Cluster(env, tiers=[ssd_tier, nvram_tier])
app1 = Application(env, name="app1", compute=[0, 10, 25], read=[1e9, 0, 2e9], write=[0, 5e9, 0], data=data)
app2 = Application(env, name="app2", compute=[0, 15], read=[0, 0], write=[0, 0], data=data)
app3 = Application(env, name="app3", compute=[10, 25], read=[0, 0], write=[0, 0], data=data)
env.process(app1.run(cluster, tiers=[0, 1, 0]))
env.process(app2.run(cluster, tiers=[0, 1]))
env.process(app3.run(cluster, tiers=[0, 1]))
env.run()


fig = display_apps(data, height=900)
fig.show()
fig = display_cluster(data, cluster)
fig.show()



2022-04-13 15:48:02.525 | INFO     | phase:run:120 - (App app1) - Start Read I/O Phase with volume = 953.67 MB at 0
2022-04-13 15:48:02.526 | INFO     | phase:run:121 - (App app1) - Read(ing) I/O with bandwidth = 210.0 MB/s
2022-04-13 15:48:02.527 | INFO     | phase:run:60 - (App app2) - Start computing phase at 0 with 1 requested cores
2022-04-13 15:48:02.528 | DEBUG    | phase:monitor:15 - --Monitoring| app: app1 | type: read | cpu_usage: 1 | t_start: 0 | t_end: 3.8880789567986955 | bandwidth: 257.1964229922337 | phase_duration: 3.8880789567986955 | volume: 1000000000.0 | tiers: ['SSD', 'NVRAM'] | data_placement: {'placement': 'SSD'} | tier_level: {'SSD': 1000000000.0, 'NVRAM': 0} 
2022-04-13 15:48:02.529 | INFO     | phase:run:140 - (App app1) - End Read I/O Phase at 3.8880789567986955
2022-04-13 15:48:02.530 | INFO     | phase:run:60 - (App app3) - Start computing phase at 3.8880789567986955 with 1 requested cores
2022-04-13 15:48:02.531 | DEBUG    | phase:monitor:15 - --Monitoring

----------app1----------
----------app2----------
----------app3----------


Debugging session

In [4]:
logger.remove()
env = simpy.Environment()
data = simpy.Store(env)
nvram_bandwidth = {'read':  {'seq': 780, 'rand': 760},
                           'write': {'seq': 515, 'rand': 505}}
ssd_bandwidth = {'read':  {'seq': 210, 'rand': 190},
                    'write': {'seq': 100, 'rand': 100}}

ssd_tier = Tier(env, 'SSD', bandwidth=ssd_bandwidth, capacity=200e9)
nvram_tier = Tier(env, 'NVRAM', bandwidth=nvram_bandwidth, capacity=80e9)
cluster = Cluster(env, tiers=[ssd_tier, nvram_tier])
app1 = Application(env, name="app1", compute=[0, 10], read=[1e9, 0], write=[0, 5e9], data=data)
app2 = Application(env, name="app2", compute=[0, 15], read=[0, 0], write=[0, 0], delay=40, data=data)
env.process(app1.run(cluster, tiers=[0, 0]))
env.process(app2.run(cluster, tiers=[1, 1]))
env.run()

fig = display_run(data, cluster)
fig.show()


----------app1----------
----------app2----------
